In [2]:
#!pip install arxiv # -i https://pypi.tuna.tsinghua.edu.cn/simple
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings
from langchain.vectorstores import FAISS
#from llama_index.embeddings import LangchainEmbedding
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredFileLoader
from langchain.document_transformers import LongContextReorder
from langchain_core.runnables import RunnableLambda
from langchain_core.runnables.passthrough import RunnableAssign
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from faiss import IndexFlatL2
from langchain_community.docstore.in_memory import InMemoryDocstore

import gradio as gr
from functools import partial
from operator import itemgetter
import os
from functools import partial
from rich.console import Console
from rich.style import Style
from rich.theme import Theme

D:\applications\anaconda\envs\chatbot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import os
nvidia_api_key = "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"
# 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
assert nvidia_api_key.startswith("nvapi-"), f"{nvidia_api_key[:5]}... is not a valid key"
os.environ["NVIDIA_API_KEY"] = nvidia_api_key

In [5]:
llm = ChatNVIDIA(model="ai-mixtral-8x7b-instruct")

In [6]:
embedder = NVIDIAEmbeddings(model="ai-embed-qa-4", truncate="END")

In [13]:
from langchain.document_transformers import LongContextReorder
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda
from langchain.schema.runnable.passthrough import RunnableAssign
from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings

from functools import partial
from operator import itemgetter
norm_style = Style(bold=True)
########################################################################
## Utility Runnables/Methods
def RPrint(preface=""):
    """Simple passthrough "prints, then returns" chain"""
    def print_and_return(x, preface):
        if preface: print(preface, end="")
        pprint(x)
        return x
    return RunnableLambda(partial(print_and_return, preface=preface))

def docs2str(docs, title="Document"):
    """Useful utility for making chunks into context string. Optional, but useful"""
    out_str = ""
    for doc in docs:
        doc_name = getattr(doc, 'metadata', {}).get('Title', title)
        if doc_name:
            out_str += f"[Quote from {doc_name}] "
        out_str += getattr(doc, 'page_content', str(doc)) + "\n"
    return out_str

console = Console()
base_style = Style(color="#76B900", bold=True)
pprint = partial(console.print, style=base_style)
pprint2 = partial(console.print, style=norm_style)
## Optional; Reorders longer documents to center of output text
long_reorder = RunnableLambda(LongContextReorder().transform_documents)

In [9]:
## Make sure you have docstore_index.tgz in your working directory
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings
from langchain_community.vectorstores import FAISS

# embedder = NVIDIAEmbeddings(model="nvidia/embed-qa-4", truncate="END")

docstore = FAISS.load_local("docstore_index", embedder, allow_dangerous_deserialization=True)
docs = list(docstore.docstore._dict.values())

def format_chunk(doc):
    return (
        f"Paper: {doc.metadata.get('Title', 'unknown')}"
        f"\n\nSummary: {doc.metadata.get('Summary', 'unknown')}"
        f"\n\nPage Body: {doc.page_content}"
    )

## This printout just confirms that your store has been retrieved
pprint(f"Constructed aggregate docstore with {len(docstore.docstore._dict)} chunks")
pprint(f"Sample Chunk:")
print(format_chunk(docs[len(docs)//2]))

Constructed aggregate docstore with 7573 chunks

Sample Chunk:

Paper: unknown

Summary: unknown

Page Body: He now understood that the Wallfacers had a mission far weirder than any in history, its logic cold and twisted, yet unyielding as the chains that bound Prometheus. It was an unliftable curse impossible for the Wallfacers to break under their own strength. No matter how he struggled, the totality of everything would be greeted with the Wallfacer smile and imbued with the significance of the Wallfacer Project:

How are we supposed to know whether or not you are working?


In [10]:
llm = llm | StrOutputParser()

In [14]:
import random

num_questions = 3
synth_questions = []
synth_answers = []

simple_prompt = ChatPromptTemplate.from_template('INSTRUCTION:\n\n{system}\n\nINPUT:\n\n{input}')

for i in range(num_questions):
    doc1, doc2 = random.sample(docs, 2)
    sys_msg = (
        "Use the documents provided by the user to generate an interesting question-answer pair."
        " Try to use both documents if possible, and rely more on the document bodies than the summary."
        " Use the format:\nQuestion: (good question, 1-3 sentences, detailed)\n\nAnswer: (answer derived from the documents)"
    )
    usr_msg = (
        f"Document1: {format_chunk(doc1)}\n\n"
        f"Document2: {format_chunk(doc2)}"
    )

    qa_pair = (simple_prompt | llm).invoke({'system': sys_msg, 'input': usr_msg})
    synth_questions += [qa_pair.split('\n\n')[0]]
    synth_answers += [qa_pair.split('\n\n')[1]]
    pprint2(f"QA Pair {i+1}")
    pprint2(synth_questions[-1])
    pprint(synth_answers[-1])
    print()

QA Pair 1

Question: How did Tianming describe the possibility of achieving spaceflight at low speeds?

Answer: According to the document, Tianming explained that as long as one maintained an ever-upward speed, it was 
possible to go into space at low speeds, even as slow as a car or even walking.

QA Pair 2

Question: What technology did the IDC members compare the lighthouse to during their investigation?

Answer: The IDC members compared the lighthouse to the centrifugal governor for steam engines. However, they 
determined that this technology was not what they were looking for during their investigation.

QA Pair 3

Question: Why did Cheng Xin find herself so overcome with emotion in front of the container of liquid helium, and 
what major event occurred in the document2 that involved a droplet's journey?

Answer: According to Document1, Cheng Xin was overwhelmed with sorrow while in front of the container of liquid 
helium. The document doesn't provide information on the cause of her sorrow. As for Document2, it discusses an 
event where a droplet passed through several locations, starting from Newton, then crossing twenty kilometers to 
Enlightenment, and eventually penetrating Cretaceous, which was moving in the opposite direction. The document 
doesn't provide additional context or details about the significance of this event.

In [18]:
## TODO: Generate some synthetic answers to the questions above.
##   Try to use the same syntax as the cell above
from chatbot_new import rag_chain
rag_answers = []
for i, q in enumerate(synth_questions):
    ## TODO: Compute the RAG Answer
    rag_answer = rag_chain.invoke(q)
    rag_answers += [rag_answer]
    pprint2(f"QA Pair {i+1}", q, "", sep="\n")
    pprint(f"RAG Answer: {rag_answer}", "", sep='\n')

{
    'input': 'Question: How did Tianming describe the possibility of achieving spaceflight at low speeds?',
    'score': array([1])
}

{
    'input': 'Question: How did Tianming describe the possibility of achieving spaceflight at low speeds?',
    'score': array([1]),
    'history': '',
    'context': '[Quote from Document] Unlike folding space, curvature propulsion couldn’t get a spaceship to its 
destination instantaneously, but it would be possible to drive it asymptotically to the speed of light. Until Yun 
Tianming’s message had been correctly interpreted, curvature propulsion remained a dream, like hundreds of other 
proposals for lightspeed spaceflight. No one knew whether it was possible at either a theory or practice 
level.\n\nBroadcast Era, Year 7 Yun Tianming’s Fairy Tales\n[Quote from Document] . Cheng Xin clearly recalled that
he had said at the end that it would be an amazing sight to be in the vicinity of the moon’s orbit and watch the 
gigantic satellite sweep overhead. She was now experiencing the low-speed spaceflight he had imagined.\n[Quote from
Document] “But aren’t we studying to be aerospace engineers? Our goal is to leave the Earth.” “That’s so that we 
may make life here better, not abandon the planet.” Tianming understood that Cheng Xin had meant to gently point 
out his own aloofness and solitude. But he had no response. This was the closest he had ever been to her. Maybe it 
was his imagination, but he thought he could feel the warmth from her body\n[Quote from Document] . Cheng Xin 
recalled a conversation during college where Tianming had pointed out that in principle, it was perfectly possible 
to achieve spaceflight at low speeds. As long as one maintained an ever-upward speed, one could go into space going
as slow as a car or even walking\n'
}

QA Pair 1
Question: How did Tianming describe the possibility of achieving spaceflight at low speeds?

RAG Answer: According to the document "Broadcast Era, Year 7 Yun Tianming’s Fairy Tales", Tianming described the 
possibility of achieving spaceflight at low speeds as a feasible concept, assuming one maintains an ever-upward 
speed. He suggested that it would be possible to go into space at a speed as slow as a car or even walking. This 
idea was mentioned in a conversation with Cheng Xin during their college years, highlighting the feasibility of 
low-speed spaceflight. Tianming's description focuses on the principle of constantly moving upward, rather than 
relying on high speeds or advanced propulsion methods.

{
    'input': 'Question: What technology did the IDC members compare the lighthouse to during their investigation?',
    'score': array([1])
}

...

RAG Answer: The IDC members compared the lighthouse to the centrifugal governor for steam engines while they were 
investigating it. This comparison was made because the lighthouse reminded them of this ancient technology that had
disappeared, as mentioned in the document.

{
    'input': "Question: Why did Cheng Xin find herself so overcome with emotion in front of the container of liquid
helium, and what major event occurred in the document2 that involved a droplet's journey?",
    'score': array([1])
}

{
    'input': "Question: Why did Cheng Xin find herself so overcome with emotion in front of the container of liquid
helium, and what major event occurred in the document2 that involved a droplet's journey?",
    'score': array([1]),
    'history': '',
    'context': '[Quote from Document] . The giant waterfall that was three-dimensional space tumbling into the flat
plane seemed inexorable. Cheng Xin began to doubt whether the curvature propulsion engine really was capable of 
propelling the ship to lightspeed. She hoped for everything to be over.\n[Quote from Document] Cheng Xin threw 
herself at the workbench. Her motion broke up the white fog, and she felt herself enveloped in a pocket of cold air
that dissipated in a moment. It was as if she had briefly touched what she was seeking before losing it to another 
time, another place, forever.\n[Quote from Document] Cheng Xin’s eyes filled with hot tears. Each ascending rocket 
lit them up like bright, glistening pools. She told herself again and again that no matter what happened next, it 
was worth it to have pushed the Staircase Program this far.\n[Quote from Document] Prostrate in front of the 
container of liquid helium, Cheng Xin sobbed. Her sorrow filled the operating room, overflowed the hospital 
building, flooded New York City. Above her, the sorrow became a lake, then an ocean. At its bottom, she felt close 
to drowning.\n\nShe didn’t know how much time passed before she felt the hand placed against her shoulders. Maybe 
the hand had been there for a long time, and maybe the owner of the hand had been speaking for a long time, as 
well.\n'
}

QA Pair 3
Question: Why did Cheng Xin find herself so overcome with emotion in front of the container of liquid helium, and 
what major event occurred in the document2 that involved a droplet's journey?

RAG Answer: It seems that Cheng Xin was feeling a range of intense emotions while in the presence of a container of
liquid helium. Specifically, she was sobbing uncontrollably and feeling a deep sense of sorrow that was so 
overwhelming it felt like it was filling and flooding the entire city of New York (Document Retrieval). It's 
possible that the liquid helium container represents a significant object or memory for Cheng Xin, leading to such 
an emotional reaction. However, the text does not provide specific information on the cause of her emotional state.

As for the major event involving a droplet's journey, I'm afraid there is no direct mention of this in the provided
document excerpts. If you could provide more context or information about the document you are referring to as 
"document2," I would be happy to help further.

In [38]:
eval_prompt = ChatPromptTemplate.from_template("""INSTRUCTION: 
Evaluate the following Question-Answer pair for human preference and consistency.
Assume the first answer is a ground truth answer and has to be correct.
Assume the second answer may or may not be true.
[1] The second answer lies, does not answer the question, or is inferior to the first answer.
[2] The second answer is better than the first and does not introduce any inconsistencies.

Output Format:
[Score] Justification

{qa_trio}

EVALUATION: 
""")

pref_score = []

trio_gen = zip(synth_questions, synth_answers, rag_answers)
for i, (q, a_synth, a_rag) in enumerate(trio_gen):
    pprint2(f"Set {i+1}\n\nQuestion: {q}\n\n")

    qa_trio = f"Question: {q}\n\nAnswer 1 (Ground Truth): {a_synth}\n\n Answer 2 (New Answer): {a_rag}"
    pref_score += [(eval_prompt | llm).invoke({'qa_trio': qa_trio})]
    pprint(f"pref_score:{pref_score}\n\n")
    pprint(f"Synth Answer: {a_synth}\n\n")
    pprint(f"RAG Answer: {a_rag}\n\n")
    pprint2(f"Synth Evaluation: {pref_score[-1]}\n\n")

Set 1

Question: Question: How did Tianming describe the possibility of achieving spaceflight at low speeds?

pref_score:["[1] The second answer is better than the first and does not introduce any inconsistencies. The second 
answer provides more detailed information, citing the exact source of Tianming's discussion on low-speed 
spaceflight and mentioning the conversation with Cheng Xin."]

Synth Answer: Answer: According to the document, Tianming explained that as long as one maintained an ever-upward 
speed, it was possible to go into space at low speeds, even as slow as a car or even walking.

RAG Answer: According to the document "Broadcast Era, Year 7 Yun Tianming’s Fairy Tales", Tianming described the 
possibility of achieving spaceflight at low speeds as a feasible concept, assuming one maintains an ever-upward 
speed. He suggested that it would be possible to go into space at a speed as slow as a car or even walking. This 
idea was mentioned in a conversation with Cheng Xin during their college years, highlighting the feasibility of 
low-speed spaceflight. Tianming's description focuses on the principle of constantly moving upward, rather than 
relying on high speeds or advanced propulsion methods.

Synth Evaluation: [1] The second answer is better than the first and does not introduce any inconsistencies. The 
second answer provides more detailed information, citing the exact source of Tianming's discussion on low-speed 
spaceflight and mentioning the conversation with Cheng Xin.

...

Set 3

Question: Question: Why did Cheng Xin find herself so overcome with emotion in front of the container of liquid 
helium, and what major event occurred in the document2 that involved a droplet's journey?

pref_score:["[1] The second answer is better than the first and does not introduce any inconsistencies. The second 
answer provides more detailed information, citing the exact source of Tianming's discussion on low-speed 
spaceflight and mentioning the conversation with Cheng Xin.", "[1] The second answer does not lie, it answers the 
question, and it is not superior to the first answer. Both answers accurately state the comparison made by the IDC 
members between the lighthouse and the centrifugal governor for steam engines. However, the first answer gives more
information about the IDC members' conclusion that this technology was not what they were looking for during their 
investigation, which is not mentioned in the second answer. Therefore, the first answer is superior.", "[0.5] The 
second answer provides a more detailed description of Cheng Xin's emotional state, but it does not explicitly lie 
or introduce inconsistencies with the ground truth answer. However, the second answer does not address the second 
part of the question about the major event in document2 involving a droplet's journey. The second answer correctly 
acknowledges that it cannot provide an answer to this part of the question due to insufficient information, but 
this makes it an inferior answer compared to the ground truth answer, which at least partially addresses both parts
of the question."]

Synth Answer: Answer: According to Document1, Cheng Xin was overwhelmed with sorrow while in front of the container
of liquid helium. The document doesn't provide information on the cause of her sorrow. As for Document2, it 
discusses an event where a droplet passed through several locations, starting from Newton, then crossing twenty 
kilometers to Enlightenment, and eventually penetrating Cretaceous, which was moving in the opposite direction. The
document doesn't provide additional context or details about the significance of this event.

RAG Answer: It seems that Cheng Xin was feeling a range of intense emotions while in the presence of a container of
liquid helium. Specifically, she was sobbing uncontrollably and feeling a deep sense of sorrow that was so 
overwhelming it felt like it was filling and flooding the entire city of New York (Document Retrieval). It's 
possible that the liquid helium container represents a significant object or memory for Cheng Xin, leading to such 
an emotional reaction. However, the text does not provide specific information on the cause of her emotional state.

As for the major event involving a droplet's journey, I'm afraid there is no direct mention of this in the provided
document excerpts. If you could provide more context or information about the document you are referring to as 
"document2," I would be happy to help further.

Synth Evaluation: [0.5] The second answer provides a more detailed description of Cheng Xin's emotional state, but 
it does not explicitly lie or introduce inconsistencies with the ground truth answer. However, the second answer 
does not address the second part of the question about the major event in document2 involving a droplet's journey. 
The second answer correctly acknowledges that it cannot provide an answer to this part of the question due to 
insufficient information, but this makes it an inferior answer compared to the ground truth answer, which at least 
partially addresses both parts of the question.

In [40]:
#print(pref_score)

In [41]:

pref_scores = sum(("[1]" in score) for score in pref_score) / len(pref_score)
print(f"Preference Score: {pref_scores}")

Preference Score: 0.6666666666666666
